### Retrieve global coins blockchain data

In [202]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash_eh_s' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] / 1e18 ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df['index_price_btc_eh_day'] = global_df.daily_emmisions_btc / global_df.nethash_eh_s # BTC / EH / day
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockchain data:')
# print(df)
global_df

Global blockchain data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash_eh_s,blocks_per_day,block_reward_btc,daily_emmisions_btc,index_price_btc_eh_day
BCH,570.0000,3.125000,3.468185e-03,SHA-256,3.063971,151.578947,0.010838,1.642824,0.536175
BTC,600.0000,3.152160,1.000000e+00,SHA-256,817.241157,144.000000,3.152160,453.911064,0.555419
DOGE,63.0000,10000.000000,2.689457e-06,Scrypt,0.002417,1371.428571,0.026895,36.883987,15261.250977
KAS,0.9997,65.561966,1.009835e-06,kHeavyHash,1.426092,86425.927778,0.000066,5.721984,4.012353
LTC,158.0000,6.250000,1.264000e-03,Scrypt,0.002075,546.835443,0.007900,4.320000,2081.805466
RVN,61.0000,2500.000000,1.515690e-07,KawPow,0.000005,1416.393443,0.000379,0.536703,99990.857467


### Retrieve NiceHash marketplace algorithms data

In [203]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
nh_market_df = pd.DataFrame(index=index)
nh_market_df['hashrate_eh_s'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / 1e18 # EH / s
nh_market_df['paying_btc_eh_day'] = algo_data_df['paying'] / sats2btc  * 1e18 # BTC/EH/day
nh_market_df['paying_btc_eh_day_$'] = (nh_market_df.paying_btc_eh_day * btc_exchange_rate).map('${:,.2f}'.format)
nh_market_df['daily_consumption_btc'] = nh_market_df.hashrate_eh_s * nh_market_df.paying_btc_eh_day
print('Nicehash marketplace data:')
# print(market_data_df)
nh_market_df

Nicehash marketplace data:


,hashrate_eh_s,paying_btc_eh_day,paying_btc_eh_day_$,daily_consumption_btc
SCRYPT,1.254825e-04,17842.802607,"$1,713,016,999.22",2.238960
SHA256ASICBOOST,7.017221e+00,0.538477,"$51,697.08",3.778614
KAWPOW,8.369037e-07,97868.856572,"$9,396,002,337.48",0.081907
KHEAVYHASH,5.477239e-02,3.855552,"$370,156.33",0.211178


### Calculate Nicehash max daily and max hourly consumption rate

In [204]:
max_market_utilisation = 0.76

nh_limits_df = pd.DataFrame(index=nh_market_df.index)
nh_limits_df['max_hashrate_eh_s'] = nh_market_df.hashrate_eh_s * max_market_utilisation
nh_limits_df['max_daily_consumption_btc'] = nh_market_df.daily_consumption_btc * max_market_utilisation
nh_limits_df['max_hourly_consumption_btc'] = nh_limits_df.max_daily_consumption_btc / 24
nh_limits_df['max_hourly_consumption_$'] = (nh_limits_df.max_hourly_consumption_btc * btc_exchange_rate).map('${:,.2f}'.format)
nh_limits_df

,max_hashrate_eh_s,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_$
SCRYPT,9.536671e-05,1.701609,0.070900,"$6,806.87"
SHA256ASICBOOST,5.333088e+00,2.871747,0.119656,"$11,487.71"
KAWPOW,6.360468e-07,0.062249,0.002594,$249.01
KHEAVYHASH,4.162702e-02,0.160495,0.006687,$642.02


### Calculate Nicehash hashrate to global hashrate ratio
Cap Nicehash hashrate to 15% of global hashrate

In [205]:
import numpy as np

max_allowed_coin_market_hp_share = 0.15

reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['nethash_eh_s'] = global_df['nethash_eh_s']
reachable_df.loc['BCH', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['BTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['DOGE', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['LTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['KAS', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KHEAVYHASH', 'max_hashrate_eh_s']
reachable_df.loc['RVN', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KAWPOW', 'max_hashrate_eh_s']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate_eh_s'] / reachable_df['nethash_eh_s']  
reachable_df['nh_hashrate_eh_s_capped'] = np.minimum(reachable_df.nh_to_global_ratio, max_allowed_coin_market_hp_share) * reachable_df['nethash_eh_s']

reachable_df

,algorithm,nethash_eh_s,nh_hashrate_eh_s,nh_to_global_ratio,nh_hashrate_eh_s_capped
BCH,SHA-256,3.063971,5.333088e+00,1.740581,4.595956e-01
BTC,SHA-256,817.241157,5.333088e+00,0.006526,5.333088e+00
DOGE,Scrypt,0.002417,9.536671e-05,0.039459,9.536671e-05
KAS,kHeavyHash,1.426092,4.162702e-02,0.029190,4.162702e-02
LTC,Scrypt,0.002075,9.536671e-05,0.045957,9.536671e-05
RVN,KawPow,0.000005,6.360468e-07,0.118499,6.360468e-07


### Calculate burn rate needed for specific amount to be spent in the time period
```
B - burn rate
A - amount
P - price
D - duration

B = A / P * D
```

In [206]:
min_order_amount_btc = 0.0001 # BTC
order_duration_s = 3600  # 1 hour

speed_df = pd.DataFrame(index=nh_market_df.index)
speed_df['amount_btc'] = min_order_amount_btc
speed_df['duration_s'] = order_duration_s
speed_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
speed_df['speed_eh_s'] = speed_df.amount_btc / (speed_df.paying_btc_eh_s * speed_df.duration_s)
speed_df

,amount_btc,duration_s,paying_btc_eh_s,speed_eh_s
SCRYPT,0.0001,3600,0.206514,1.345080e-07
SHA256ASICBOOST,0.0001,3600,0.000006,4.457012e-03
KAWPOW,0.0001,3600,1.132741,2.452261e-08
KHEAVYHASH,0.0001,3600,0.000045,6.224789e-04


### Calculate amount needed to buy speed for the time period
```
B - burn rate
A - amount
P - price
D - duration

A = B * P * D 
```

In [207]:
speed_eh_s = 1 # EH / s
order_duration_s = 3600  # 1 hour

amount_df = pd.DataFrame(index=nh_market_df.index)
amount_df['speed_eh_s'] = speed_eh_s
amount_df['duration_s'] = order_duration_s
amount_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
amount_df['amount_btc'] = amount_df.speed_eh_s * amount_df.paying_btc_eh_s * amount_df.duration_s
amount_df

,speed_eh_s,duration_s,paying_btc_eh_s,amount_btc
SCRYPT,1,3600,0.206514,743.450109
SHA256ASICBOOST,1,3600,0.000006,0.022437
KAWPOW,1,3600,1.132741,4077.869024
KHEAVYHASH,1,3600,0.000045,0.160648
